In this step, I prepared the environment for the BERT model.
I imported required libraries, checked GPU availability, and set random seeds to ensure reproducible results.This setup ensures stable and efficient training of the BERT model.



In [ ]:
import torch
import random
import numpy as np
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

#  Ensuring GPU is available for 90%+ performance
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Step 1.2: Reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)


Using device: cuda


I prepared the IMDb dataset for the BERT model.
I used the BERT tokenizer to convert text into tokens using WordPiece tokenization.All reviews were padded or truncated to a fixed length and converted into PyTorch tensors for training.

In [ ]:
# Loading Full IMDb Dataset
dataset = load_dataset("imdb")

#  BERT WordPiece Tokenization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):

    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Here, I loaded a pretrained BERT model for sequence classification.
Since the task is binary sentiment analysis, I used two output labels.
The pretrained language knowledge is reused, and the model is fine-tuned on IMDb reviews.

In [ ]:
# : Loading Pretrained BERT
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.to(device)
# The base layers stay mostly the same but adapt to IMDb style.

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
classifier.bias                            | MISSING    | 
classifier.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In this step, I defined a custom metric function to evaluate accuracy, precision, recall, and F1-score.I then configured the training arguments for fine-tuning BERT, including learning rate, batch size, number of epochs, and regularization.The model is evaluated after every epoch, and the best-performing model is automatically selected.

In [ ]:
#  Metric Calculation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions),
        "precision": precision_score(labels, predictions),
        "recall": recall_score(labels, predictions)
    }

# Optimized Training Arguments
training_args = TrainingArguments(
    output_dir="./bert_final_model",
    eval_strategy="epoch",        # cheaks performance3 after every steps
    save_strategy="epoch",
    learning_rate=2e-5,           # Recommended fine-tuning rate
    per_device_train_batch_size=16, #  optimized for gpu memory
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,  # selects Best model
    report_to="none"
)

#  Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics
)

#  Executing TRAINING
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.244059,0.231128,0.908440,0.904342,0.946714,0.865600
2,0.152126,0.268976,0.922880,0.923071,0.920793,0.925360


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.244059,0.231128,0.908440,0.904342,0.946714,0.865600
2,0.152126,0.268976,0.922880,0.923071,0.920793,0.925360
3,0.087976,0.318015,0.924080,0.924551,0.918853,0.930320


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'bert.encoder.layer.1.attention.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.1.output.LayerNorm.bias', 'bert.encoder.layer.2.attention.output.LayerNorm.weight', 'bert.encoder.layer.2.attention.output.LayerNorm.bias', 'bert.encoder.layer.2.output.LayerNorm.weight', 'bert.encoder.layer.2.output.LayerNorm.bias', 'bert.encoder.layer.3.attention.output.LayerNorm.weight', 'bert.encoder.layer.3.attention.output.LayerNorm.bias', 'bert.encoder.layer.3.output.LayerNorm.weight', 'bert.encoder.layer.3.output.LayerNorm.bias', 'bert.encoder.layer.4.attention.output.La

TrainOutput(global_step=4689, training_loss=0.17345436317474763, metrics={'train_runtime': 4767.4304, 'train_samples_per_second': 15.732, 'train_steps_per_second': 0.984, 'total_flos': 9866664576000000.0, 'train_loss': 0.17345436317474763, 'epoch': 3.0})

Finally, I saved the trained BERT model along with its tokenizer for future use and deployment.I also created a comparative report to evaluate model performance.The results clearly show that BERT significantly outperforms the custom LSTM, achieving over 92% accuracy and better F1-score and recall.This demonstrates the effectiveness of transformer-based models for sentiment analysis.

In [ ]:
# 1. Save the Final Model and Tokenizer
model.save_pretrained("./bert_final_submission")
tokenizer.save_pretrained("./bert_final_submission")

# 2. Final Comparative Data
import pandas as pd

final_data = {
    "Model": ["Custom LSTM (Project 1)", "BERT-base (Project 2)"],
    "Accuracy": ["~72.00%", "92.41%"],
    "F1-Score": ["0.7100", "0.9245"],
    "Recall": ["0.7000", "0.9303"],
    "Status": ["Baseline", "Superior (SOTA)"]
}

df_final = pd.DataFrame(final_data)
print("\n" + "="*50)
print("       OFFICIAL PROJECT COMPARISON REPORT")
print("="*50)
print(df_final.to_string(index=False))
print("="*50)

print("\nSuccess: Model saved ")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]


       OFFICIAL PROJECT COMPARISON REPORT
                  Model Accuracy F1-Score Recall          Status
Custom LSTM (Project 1)  ~72.00%   0.7100 0.7000        Baseline
  BERT-base (Project 2)   92.41%   0.9245 0.9303 Superior (SOTA)

Success: Model saved 


In [ ]:
import shutil

shutil.make_archive('bert_project_files', 'zip', './bert_final_submission')

from google.colab import files

files.download('bert_project_files.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>